In [1]:
import matplotlib.pyplot as plt
# plt.switch_backend('agg')
import pandas as pd
import geopandas as gpd
import numpy as np
import seaborn as sns
import json
import networkx as nx
import time
import os
import csv
import networkx as nx
import math
# import gerrychain


from gerrychain import Graph, Partition, Election
from gerrychain.updaters import Tally, cut_edges
from gerrychain import MarkovChain
from gerrychain.constraints import single_flip_contiguous
from gerrychain.proposals import propose_random_flip
from gerrychain.accept import always_accept
from gerrychain import (GeographicPartition, Partition, Graph, MarkovChain,
                        proposals, updaters, constraints, accept, Election)
from gerrychain.constraints.validity import within_percent_of_ideal_population
from gerrychain.metrics import mean_median,efficiency_gap
from gerrychain.proposals import recom
from functools import partial
from gerrychain.tree import recursive_tree_part
from gerrychain.tree import bipartition_tree

from tqdm import tqdm

In [2]:
graph = Graph.from_file("/Users/jnc/Research and Presentations/Gerrymandering and elections/Colorado analysis/Data files/2018/CO_Shapefile_2018_from_Haley_Jan2020/co_precincts.shp")

In [3]:
df = gpd.read_file("/Users/jnc/Research and Presentations/Gerrymandering and elections/Colorado analysis/Data files/2018/CO_Shapefile_2018_from_Haley_Jan2020/co_precincts.shp")

### Display first few rows of data frame:

In [4]:
pd.set_option('display.max_columns', None)
df.head()

,COUNTYFP,VTDST,NAME,CD116FP,SLDUST,SLDLST,PRECID,AG18D,AG18R,SOS18D,SOS18R,TRE18D,TRE18R,GOV18D,GOV18R,REG18D,REG18R,USH18D,USH18R,TOTPOP,NH_WHITE,NH_BLACK,NH_AMIN,NH_ASIAN,NH_NHPI,NH_OTHER,NH_2MORE,HISP,H_WHITE,H_BLACK,H_AMIN,H_ASIAN,H_NHPI,H_OTHER,H_2MORE,VAP,HVAP,WVAP,BVAP,AMINVAP,ASIANVAP,NHPIVAP,OTHERVAP,2MOREVAP,geometry
0,005,005230,Arapahoe 230,01,031,009,1310903230,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"POLYGON ((508234.924 4386190.022, 508246.088 4..."
1,005,005256,Arapahoe 256,06,026,003,2147483647,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"POLYGON ((512663.103 4386046.307, 512694.687 4..."
2,005,005359,Arapahoe 359,04,029,056,2147483647,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"POLYGON ((532402.077 4393729.843, 532403.119 4..."
3,005,005561,Arapahoe 561,06,028,036,2147483647,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"POLYGON ((521125.938 4392491.967, 521702.039 4..."
4,031,031745,Denver 745,01,033,007,1330716745,1084,303,1073,316,1046,334,1137,272,1050,254,1121,264,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"POLYGON ((511474.738 4404493.245, 511440.417 4..."


### We won't really need all these variables just to generate random seeds, but we do need to at least extract the total population of the state, and this sequence of events does that, among other things.

In [6]:
global unique_label 
unique_label = "PRECID"  #Unique identifier for the voting district
global population_col
population_col = "TOTPOP" 
global district_col
district_col = "SLDLST" #Change for: state house (SLDLST) or house sentate district (SLDUST) or congressional (CD116FP)
global county_col
county_col = "COUNTYFP"

In [7]:
#Input the elections you are interested in

election_names = [
    "GOV", "AG", "SEC", "TRE", "REG"
]

election_columns = [
    ["GOV18D", "GOV18R"], ["AG18D","AG18R"], ["SOS18D","SOS18R"],["TRE18D","TRE18R"],["REG18D","REG18R"]
]

In [8]:
num_elections = len(election_names)
num_districts = 65 #Can change this to look at state house or senate districts

In [9]:
elections = [
    Election(
        election_names[i],
        {"Democratic": election_columns[i][0], "Republican": election_columns[i][1]},
    )
    for i in range(num_elections)
]

In [10]:
updaters = {"population": updaters.Tally(population_col, alias="population"),
            "cut_edges": cut_edges}

election_updaters = {election.name: election for election in elections}

updaters.update(election_updaters)

### Compute the total population for the state, and the ideal district population at each level:

In [11]:
enacted_plan = GeographicPartition(graph, assignment=district_col, updaters=updaters)
total_pop = sum(enacted_plan["population"].values())

ideal_pop_CD = int(total_pop/7)   # 7 Congressional seats
ideal_pop_SUD = int(total_pop/35)   # 35 State Senate seats   
ideal_pop_SLD = int(total_pop/65)   # 65 State House seats



### Check that we got these population numbers correct:

In [12]:
print(total_pop)
print(ideal_pop_CD)
print(ideal_pop_SUD)
print(ideal_pop_SLD)

5027604
718229
143645
77347


### Create a list of 20 random seeds at each level; then we will write these seeds to json files.
### Use strict population tolerance (within 0.5% of ideal pop) for Congressional districts and less strict (within 2.5% of ideal pop) for state districts

### I would have liked to do 30, but for some bizarre reason the procedure gets stuck trying to create the 24th seed.  So I decided to settle for 20.

In [13]:
seeds_SLD = []

for i in tqdm(range(20)):
    seeds_SLD.append(recursive_tree_part(graph, range(65), ideal_pop_SLD, population_col, .025, 1))
    #print(i)

 75%|███████▌  | 15/20 [05:33<01:51, 22.22s/it]


KeyboardInterrupt: 

It got stuck!

In [14]:
len(seeds_SLD)

15

We'll make do with that.

In [15]:
basename = "./Outputs/RandomSeedsSLD/seed_SLD_"

for i in range(15):
   with open(basename+str(i).zfill(2)+".json", 'w') as file_to_write:
    json.dump(dict(seeds_SLD[i]), file_to_write)
 